In [2]:
import time
import re
import numpy as np
import pandas as pd
import warnings;warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [3]:
df_train = pd.read_csv('train.csv',lineterminator='\n')
df_test = pd.read_csv('test.csv',lineterminator='\n')

In [6]:
df_train['label'] = df_train['label'].map({'Negative':0,'Positive':1})
df_train.head()

,ID,review,label
0,1,Jo bhi ap se tou behtar hoon,0
1,2,ya Allah meri sister Affia ki madad farma,1
2,3,Yeh khud chahta a is umar main shadi krna. ha...,0
3,4,Tc ? Apky mun xe exe alfax achy nae lgty 😒💃,0
4,5,Good,1


In [7]:
df_train.isnull().sum()

ID        0
review    0
label     0
dtype: int64

In [8]:
df_train.isnull().sum()

ID        0
review    0
label     0
dtype: int64

In [9]:
df_train['label'].value_counts()

1    3361
0    2967
Name: label, dtype: int64

In [10]:
numpy_array = df_train.as_matrix()
numpy_array_test = df_test.as_matrix()
numpy_array[:4]

array([[1, 'Jo bhi ap se tou behtar hoon', 0],
       [2, 'ya Allah meri sister Affia ki madad farma', 1],
       [3, 'Yeh khud chahta a is umar main shadi krna.  had ogi', 0],
       [4, 'Tc ? Apky mun xe exe alfax achy nae lgty 😒💃', 0]],
      dtype=object)

In [11]:
numpy_array_test[115]

array([116, 'Kisi Roz Jo Meri Gali Se Ishq Ka Janaza Nikle '],
      dtype=object)

In [12]:
#two commom ways to clean data
def cleaner(word):
  word = re.sub(r'\#\.', '', word)
  word = re.sub(r'\n', '', word)
  word = re.sub(r',', '', word)
  word = re.sub(r'\-', ' ', word)
  word = re.sub(r'\.', '', word)
  word = re.sub(r'\\', ' ', word)
  word = re.sub(r'\\x\.+', '', word)
  word = re.sub(r'\d', '', word)
  word = re.sub(r'^_.', '', word)
  word = re.sub(r'_', ' ', word)
  word = re.sub(r'^ ', '', word)
  word = re.sub(r' $', '', word)
  word = re.sub(r'\?', '', word)
  word = re.sub(r'é', '', word)
  word = re.sub(r'§', '', word)
  word = re.sub(r'¦', '', word)
  word = re.sub(r'æ', '', word)
  word = re.sub(r'\d+', '', word)
  word = re.sub('(.*?)\d+(.*?)', '', word)
  return word.lower()
def hashing(word):
  word = re.sub(r'ain$', r'ein', word)
  word = re.sub(r'ai', r'ae', word)
  word = re.sub(r'ay$', r'e', word)
  word = re.sub(r'ey$', r'e', word)
  word = re.sub(r'ie$', r'y', word)
  word = re.sub(r'^es', r'is', word)
  word = re.sub(r'a+', r'a', word)
  word = re.sub(r'j+', r'j', word)
  word = re.sub(r'd+', r'd', word)
  word = re.sub(r'u', r'o', word)
  word = re.sub(r'o+', r'o', word)
  word = re.sub(r'ee+', r'i', word)
  if not re.match(r'ar', word):
    word = re.sub(r'ar', r'r', word)
  word = re.sub(r'iy+', r'i', word)
  word = re.sub(r'ih+', r'eh', word)
  word = re.sub(r's+', r's', word)
  if re.search(r'[rst]y', 'word') and word[-1] != 'y':
    word = re.sub(r'y', r'i', word)
  if re.search(r'[bcdefghijklmnopqrtuvwxyz]i', word):
    word = re.sub(r'i$', r'y', word)
  if re.search(r'[acefghijlmnoqrstuvwxyz]h', word):
    word = re.sub(r'h', '', word)
  word = re.sub(r'k', r'q', word)
  return word

def array_cleaner(array):
  # X = array
  X = []
  for sentence in array:
    clean_sentence = ''
    words = sentence.split(' ')
    for word in words:
      clean_sentence = clean_sentence +' '+ cleaner(word)
    X.append(clean_sentence)
  return X

In [13]:
X_test = numpy_array_test[:,1]
X_test

array(['Yaqoob Memon Ki Phansi Zalimana Ghair Insani Hai 20 Saal Qaid Kaat Chukay Thay Amnesty International Ki Bharti Iqdam Ki Muzammat ',
       ' Sabit qadam rehna',
       'Good decision on ko bhi aam shehryun ki tarah Huqooq hony chahye',
       ..., 'Lanat haramdyo', 'Is episode mein koi bhi hot nai hai',
       'Mery liye dua kry Allah pak naseeb achy kry ameen'], dtype=object)

In [14]:
#test if there are nan 
counter = 1
for sentence in X_test:
    try:
        words = sentence.split(' ')
        counter+=1
    except:
        print(sentence)
        print(counter)

In [15]:
X_train = numpy_array[:, 1]
# Clean X here
X_train = array_cleaner(X_train)
X_test = array_cleaner(X_test)
y_train = numpy_array[:, 2]
X_train[:5]

[' jo bhi ap se tou behtar hoon',
 ' ya allah meri sister affia ki madad farma',
 ' yeh khud chahta a is umar main shadi krna  had ogi',
 ' tc  apky mun xe exe alfax achy nae lgty 😒💃',
 ' good']

In [16]:
print(len(X_train))
print(len(X_test))

6328
2712


In [35]:
y_train = np.array(y_train)
y_train = y_train.astype('int8')
print(y_train.shape)
y_train[:6]

(6328,)


array([0, 1, 0, 0, 1, 0], dtype=int8)

In [41]:
test1 = pd.Series(y_train)
test1.unique()

array([0, 1])

In [18]:
ngram = 2
vectorizer = TfidfVectorizer(sublinear_tf=True,ngram_range=(1, ngram), max_df=0.5)

In [19]:
X_all = X_train + X_test # Combine both to fit the TFIDF vectorization.
lentrain = len(X_train)

vectorizer.fit(X_all)
X_all = vectorizer.transform(X_all)

In [20]:
vectorizer.get_feature_names()[-5:]

['賭easar ul', '鄭h', '鄭h isnan', '鄭pwa', '鄭pwa yani']

In [21]:
X_all.shape

(9040, 113521)

In [22]:
X_train_chuli = X_all[:lentrain] # Separate back into training and test sets. 
X_test_chuli = X_all[lentrain:]

In [23]:
X_train_chuli.shape

(6328, 113521)

In [25]:
#bayesian optimization to find hyperparameter for lightgbm
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from bayes_opt import BayesianOptimization
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [68]:
def LGB_CV(
          min_data_in_leaf,
          feature_fraction,
          bagging_fraction,
         ):
    
    folds = KFold(n_splits=5, shuffle=True, random_state=2019)
    oof = np.zeros(X_train_chuli.shape[0])

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train_chuli, y_train)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(X_train_chuli[trn_idx],
                               label=y_train[trn_idx],
                               )
        val_data = lgb.Dataset(X_train_chuli[val_idx],
                               label=y_train[val_idx],
                               )
    
        param = {
            'max_depth': -1,
            'min_data_in_leaf': int(min_data_in_leaf), 
            'objective':'binary',
            'bagging_fraction':bagging_fraction,
            'feature_fraction':feature_fraction,
            'learning_rate': 0.005,
            "boosting": "gbdt",
            "bagging_freq": 5,
            "bagging_seed": 11,
            "metric": 'auc',
            "verbosity": -1
        }
    
        clf = lgb.train(param,
                        trn_data,
                        8000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds = 500)
        
        oof[val_idx] = clf.predict(X_train_chuli[val_idx],
                                   num_iteration=clf.best_iteration)
        
        del clf, trn_idx, val_idx
        
    return metrics.roc_auc_score(y_train,oof)


In [69]:
LGB_BO = BayesianOptimization(LGB_CV, {
        'min_data_in_leaf': (2, 40),
        'bagging_fraction': (0.01, 0.999),
        'feature_fraction':(0.01, 0.999)
    })

In [70]:
LGB_BO.maximize(init_points=2,n_iter=2)

|   iter    |  target   | baggin... | featur... | min_da... |
-------------------------------------------------------------
fold n°0
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.8078	valid_1's auc: 0.749847
[1000]	training's auc: 0.841791	valid_1's auc: 0.752634
[1500]	training's auc: 0.867406	valid_1's auc: 0.752817
[2000]	training's auc: 0.889175	valid_1's auc: 0.754049
Early stopping, best iteration is:
[1902]	training's auc: 0.88553	valid_1's auc: 0.755229
fold n°1
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.809128	valid_1's auc: 0.733232
Early stopping, best iteration is:
[460]	training's auc: 0.804646	valid_1's auc: 0.7365
fold n°2
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.813148	valid_1's auc: 0.716077
[1000]	training's auc: 0.847292	valid_1's auc: 0.722404
[1500]	training's auc: 0.872096	valid_1's auc: 0.724389
Early stopping, best iteration is:
[115

In [83]:
folds = KFold(n_splits=5, shuffle=True, random_state=2019)
oof = np.zeros(X_train_chuli.shape[0])
predictions = np.zeros(X_test_chuli.shape[0])
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train_chuli, y_train)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(X_train_chuli[trn_idx],
                           label=y_train[trn_idx],
                           )
    val_data = lgb.Dataset(X_train_chuli[val_idx],
                           label=y_train[val_idx],
                           )

    param = {
        'max_depth': -1,
        'min_data_in_leaf': 2, 
        'objective':'binary',
        'bagging_fraction':0.999,
        'feature_fraction':0.999,
        'learning_rate': 0.005,
        "boosting": "gbdt",
        "bagging_freq": 5,
        "bagging_seed": 11,
        "metric": 'auc',
        "verbosity": -1
    }

    clf = lgb.train(param,
                    trn_data,
                    8000,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=500,
                    early_stopping_rounds = 500)

    oof[val_idx] = clf.predict(X_train_chuli[val_idx],
                               num_iteration=clf.best_iteration)
    predictions += clf.predict(X_test_chuli, num_iteration=clf.best_iteration) / folds.n_splits

fold n°0
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.879041	valid_1's auc: 0.790074
[1000]	training's auc: 0.931967	valid_1's auc: 0.80286
[1500]	training's auc: 0.961189	valid_1's auc: 0.806492
[2000]	training's auc: 0.978269	valid_1's auc: 0.809039
[2500]	training's auc: 0.987688	valid_1's auc: 0.81074
Early stopping, best iteration is:
[2429]	training's auc: 0.986633	valid_1's auc: 0.811102
fold n°1
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.88044	valid_1's auc: 0.782168
[1000]	training's auc: 0.932543	valid_1's auc: 0.798356
[1500]	training's auc: 0.96102	valid_1's auc: 0.803757
[2000]	training's auc: 0.977631	valid_1's auc: 0.806902
[2500]	training's auc: 0.987358	valid_1's auc: 0.808384
[3000]	training's auc: 0.993203	valid_1's auc: 0.808181
Early stopping, best iteration is:
[2611]	training's auc: 0.988966	valid_1's auc: 0.808651
fold n°2
Training until validation scores don't improve for 500 

In [84]:
print(len(predictions))
predictions[:4]

2712


array([0.22984153, 0.62412493, 0.98233711, 0.79193217])

In [85]:
lgb_output = pd.DataFrame({"ID":df_test["ID"], "Pred":predictions})
lgb_output.to_csv('lgb_new.csv', index = False)